# 1. Imports

In [1]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from src.dim import Dims
from src.facts import Facts
import pandas as pd
import time
import datetime
from pyspark.sql import functions as f 
from pyspark.sql import  SparkSession, Window
from aggregate_data import DataAggregator



pd.set_option('display.max_columns', None)



# 2. Variables

In [2]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DIMS = Dims(config, fetcher)
FACTS = Facts(config, fetcher)
DATAAGGREGATOR = DataAggregator(config, fetcher)
spark = SparkSession.builder.appName("MojaSesja").master("local").getOrCreate()



25/10/12 18:31:18 WARN Utils: Your hostname, MacBook-Pro-Grzegorz.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
25/10/12 18:31:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/12 18:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/12 18:31:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# 1. Assigning points throught the season to the driver.

In [9]:
dim_session = DIMS.dim_sessions(race="Race")
fact_session_result = FACTS.fact_session_results()

In [10]:
fact_session_result['position'] = fact_session_result['position'].fillna(value=21)
dim_driver_team = DIMS.dim_driver_team()

In [11]:
dim_session

,session_key,location,date_start,date_end,session_name,country_code,country_name,year,is_current_season,key
1,7953,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023
3,7779,Jeddah,2023-03-19,2023-03-19,Race,KSA,Saudi Arabia,2023,0,Jeddah2023
5,7787,Melbourne,2023-04-02,2023-04-02,Race,AUS,Australia,2023,0,Melbourne2023
9,9070,Baku,2023-04-30,2023-04-30,Race,AZE,Azerbaijan,2023,0,Baku2023
11,9078,Miami,2023-05-07,2023-05-07,Race,USA,United States,2023,0,Miami2023
...,...,...,...,...,...,...,...,...,...,...
149,9928,Budapest,2025-08-03,2025-08-03,Race,HUN,Hungary,2025,1,Budapest2025
151,9920,Zandvoort,2025-08-31,2025-08-31,Race,NED,Netherlands,2025,1,Zandvoort2025
153,9912,Monza,2025-09-07,2025-09-07,Race,ITA,Italy,2025,1,Monza2025
155,9904,Baku,2025-09-21,2025-09-21,Race,AZE,Azerbaijan,2025,1,Baku2025


In [13]:
joined_results  = fact_session_result.merge(
    dim_session,
    on="session_key",
    how="inner"
).merge(
    dim_driver_team,
    on = ['session_key','driver_number'],
    how = "inner"
)



In [14]:
joined_results

,position,driver_number,number_of_laps,dnf,dns,dsq,duration,gap_to_leader,meeting_key,session_key,points,location,date_start,date_end,session_name,country_code,country_name,year,is_current_season,key,team_name
0,1.0,1,57.0,False,False,False,5636.736,0,1141,7953,25.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Red Bull Racing
1,2.0,11,57.0,False,False,False,5648.723,11.987,1141,7953,18.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Red Bull Racing
2,3.0,14,57.0,False,False,False,5675.373,38.637,1141,7953,15.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Aston Martin
3,4.0,55,57.0,False,False,False,5684.788,48.052,1141,7953,12.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Ferrari
4,5.0,44,57.0,False,False,False,5687.713,50.977,1141,7953,10.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Mercedes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,21.0,12,23.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Mercedes
1273,21.0,6,17.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Racing Bulls
1274,21.0,5,3.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Kick Sauber
1275,21.0,30,0.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Racing Bulls


In [16]:
joined_results = joined_results[['position','driver_number','session_key','points','year','is_current_season','key','date_end','team_name']]
spark_df = spark.createDataFrame(joined_results)
spark_df.show()

+--------+-------------+-----------+------+----+-----------------+----------+----------+---------------+
|position|driver_number|session_key|points|year|is_current_season|       key|  date_end|      team_name|
+--------+-------------+-----------+------+----+-----------------+----------+----------+---------------+
|     1.0|            1|       7953|  25.0|2023|                0|Sakhir2023|2023-03-05|Red Bull Racing|
|     2.0|           11|       7953|  18.0|2023|                0|Sakhir2023|2023-03-05|Red Bull Racing|
|     3.0|           14|       7953|  15.0|2023|                0|Sakhir2023|2023-03-05|   Aston Martin|
|     4.0|           55|       7953|  12.0|2023|                0|Sakhir2023|2023-03-05|        Ferrari|
|     5.0|           44|       7953|  10.0|2023|                0|Sakhir2023|2023-03-05|       Mercedes|
|     6.0|           18|       7953|   8.0|2023|                0|Sakhir2023|2023-03-05|   Aston Martin|
|     7.0|           63|       7953|   6.0|2023|       

In [48]:
window = Window.partitionBy("driver_number","year").orderBy("date_end").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

spark_df = spark_df.withColumn("cumulative_points", f.sum("points").over(window))

In [50]:
spark_df.show(50)

+--------+-------------+-----------+------+----+-----------------+--------------------+----------+-----------------+
|position|driver_number|session_key|points|year|is_current_season|                 key|  date_end|cumulative_points|
+--------+-------------+-----------+------+----+-----------------+--------------------+----------+-----------------+
|       1|            1|       7953|  25.0|2023|                0|          Sakhir2023|2023-03-05|             25.0|
|       2|            1|       7779|   0.0|2023|                0|          Jeddah2023|2023-03-19|             25.0|
|       1|            1|       7787|  25.0|2023|                0|       Melbourne2023|2023-04-02|             50.0|
|       2|            1|       9070|  18.0|2023|                0|            Baku2023|2023-04-30|             68.0|
|       1|            1|       9078|  26.0|2023|                0|           Miami2023|2023-05-07|             94.0|
|       1|            1|       9094|  25.0|2023|                

In [17]:
window2 = Window.partitionBy("year","team_name").orderBy("date_end")

test = spark_df.withColumn(
    "team_points",
    f.sum("points").over(window2)
)

In [18]:
test.show()

+--------+-------------+-----------+------+----+-----------------+---------------+----------+----------+-----------+
|position|driver_number|session_key|points|year|is_current_season|            key|  date_end| team_name|team_points|
+--------+-------------+-----------+------+----+-----------------+---------------+----------+----------+-----------+
|     8.0|           77|       7953|   4.0|2023|                0|     Sakhir2023|2023-03-05|Alfa Romeo|        4.0|
|    16.0|           24|       7953|   0.0|2023|                0|     Sakhir2023|2023-03-05|Alfa Romeo|        4.0|
|    13.0|           24|       7779|   0.0|2023|                0|     Jeddah2023|2023-03-19|Alfa Romeo|        4.0|
|    18.0|           77|       7779|   0.0|2023|                0|     Jeddah2023|2023-03-19|Alfa Romeo|        4.0|
|     9.0|           24|       7787|   2.0|2023|                0|  Melbourne2023|2023-04-02|Alfa Romeo|        6.0|
|    11.0|           77|       7787|   0.0|2023|                

In [4]:
DATAAGGREGATOR.get_racer_team_points(racer_team="team")

,session_key,key,year,driver_number,team_name,date_end,points,points_gained
0,7953,Sakhir2023,2023,77,Alfa Romeo,2023-03-05,4.0,4.0
1,7953,Sakhir2023,2023,24,Alfa Romeo,2023-03-05,0.0,4.0
2,7779,Jeddah2023,2023,24,Alfa Romeo,2023-03-19,0.0,4.0
3,7779,Jeddah2023,2023,77,Alfa Romeo,2023-03-19,0.0,4.0
4,7787,Melbourne2023,2023,24,Alfa Romeo,2023-04-02,2.0,6.0
...,...,...,...,...,...,...,...,...
1272,9912,Monza2025,2025,55,Williams,2025-09-07,0.0,83.0
1273,9904,Baku2025,2025,55,Williams,2025-09-21,15.0,98.0
1274,9904,Baku2025,2025,23,Williams,2025-09-21,0.0,98.0
1275,9896,Marina Bay2025,2025,55,Williams,2025-10-05,1.0,99.0
